# Finite-element mode solver

PDEs can be solved on meshed components.

For instance, a finite-element mode solver can easily be called on a component cross-section. Unlike the other mode solvers, this actually uses the component geometry instead of a hardcoded geometry.

Below, we directly compute the modes of a Gdsfactory cross-section (internally, a "uz" mesh is defined perpendicular to a straight component with the provided cross-section). We also downsample layers from the LayerStack, and both the cross-section and LayerStack can be modified prior to simulation to change the geometry. The refractive indices are interpolated from the dict `gdsfactory.pdk._ACTIVE_PDK`.

In [ ]:
%%capture
from gdsfactory.simulation.fem.mode_solver import compute_cross_section_modes
from gdsfactory.tech import LayerStack, get_layer_stack_generic
from gdsfactory.cross_section import rib

filtered_layerstack = LayerStack(
    layers={
        k: get_layer_stack_generic().layers[k]
        for k in (
            "core",
            "clad",
            "slab90",
            "box",
        )
    }
)

filtered_layerstack.layers["core"].thickness = 0.2 # Perturb the layerstack before simulating

resolutions = {}
resolutions["core"] = {"resolution": 0.02, "distance": 2}
resolutions["clad"] = {"resolution": 0.2, "distance": 1}
resolutions["box"] = {"resolution": 0.2, "distance": 1}
resolutions["slab90"] = {"resolution": 0.05, "distance": 1}

lams, basis, xs = compute_cross_section_modes(cross_section=rib(width=0.6), 
                            layerstack=filtered_layerstack,
                            wl = 1.55,
                            num_modes = 4, 
                            resolutions=resolutions,
                        )

The solver returns the effective indices (lams), FEM basis functions (basis) and eigenvectors (xs):

In [ ]:
lams, basis, xs

These can be used as inputs to other [femwell mode solver functions](https://github.com/HelgeGehring/femwell/blob/main/femwell/mode_solver.py) to inspect or analyze the modes:

In [ ]:
%%capture
from femwell import mode_solver
import numpy as np

fig = mode_solver.plot_mode(basis, np.real(xs[0]), plot_vectors=False, colorbar=True, title='E', direction='y')

In [ ]:
fig[0]

In [ ]:
%%capture
te_frac = mode_solver.calculate_te_frac(basis, xs[0])

In [ ]:
te_frac